In [1]:
import pdal
import ipyleaflet
import IPython.display
import ipyvolume.pylab as p3
import matplotlib.cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely.geometry
import scipy.spatial

import os
import requests
import json
import sys
import urllib.request


In [2]:
### Important directory paths

TM_las = '/home/jose/Documents/LiDAR/TM/GIS_data/mine_locations/alot_mines_colored.las'
py_export = '/home/jose/Github/my_projs/abm_detect/Figures/python_export/TM_lidar'

# Map of Field Area in OSM 

In [3]:
m = ipyleaflet.Map(center=(32.29, 253.3), zoom=13)
m

Map(center=[32.29, 253.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [4]:
# IS USED FOR POLYGONS! not really needed seing as ipyleaflet uses osm. only see da roads :(
dc = ipyleaflet.DrawControl()
m.add_control(dc)

In [5]:
dc.last_draw['geometry']['coordinates'][0]  # the coordinates of the drawn polygon

TypeError: 'NoneType' object is not subscriptable

# PDAL: Loading .las into memory 

In [5]:
cropper = {
    "pipeline": [ TM_las,
        {   "type":"filters.eigenvalues",
            "knn":16},
        {   "type":"filters.normal",
            "knn":16}
    ]}
pipeline = pdal.Pipeline(json.dumps(cropper))
pipeline.validate()
%time n_points = pipeline.execute()
print('Pipeline selected {} points'.format(n_points))

CPU times: user 8.59 s, sys: 72 ms, total: 8.66 s
Wall time: 8.6 s
Pipeline selected 543604 points


In [7]:
#creating numpy and pandas objects from pdal pipeline
arr = pipeline.arrays[0]
description = arr.dtype.descr
cols = [col for col, __ in description]
df = pd.DataFrame({col: arr[col] for col in cols})
df['X_0'] = df['X']
df['Y_0'] = df['Y']
df['Z_0'] = df['Z']
df['X'] = df['X'] - df['X_0'].min()
df['Y'] = df['Y'] - df['Y_0'].min()
df['Z'] = df['Z'] - df['Z_0'].min()


In [8]:
fig = p3.figure(width=500)
fig.xlabel='Y'
fig.ylabel='Z'
fig.zlabel='X'
all_points = p3.quiver(df['Y'], df['Z'], df['X'], color='red', size=.5)
p3.squarelim()
p3.show()

TypeError: quiver() missing 3 required positional arguments: 'u', 'v', and 'w'

In [21]:
description

[('X', '<f8'),
 ('Y', '<f8'),
 ('Z', '<f8'),
 ('Intensity', '<u2'),
 ('ReturnNumber', '|u1'),
 ('NumberOfReturns', '|u1'),
 ('ScanDirectionFlag', '|u1'),
 ('EdgeOfFlightLine', '|u1'),
 ('Classification', '|u1'),
 ('ScanAngleRank', '<f4'),
 ('UserData', '|u1'),
 ('PointSourceId', '<u2'),
 ('GpsTime', '<f8'),
 ('Red', '<u2'),
 ('Green', '<u2'),
 ('Blue', '<u2'),
 ('Eigenvalue0', '<f8'),
 ('Eigenvalue1', '<f8'),
 ('Eigenvalue2', '<f8'),
 ('NormalX', '<f8'),
 ('NormalY', '<f8'),
 ('NormalZ', '<f8'),
 ('Curvature', '<f8')]

In [19]:
df

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,...,Eigenvalue0,Eigenvalue1,Eigenvalue2,NormalX,NormalY,NormalZ,Curvature,X_0,Y_0,Z_0
0,187.25,65.84,48.31,33368,1,1,0,0,0,5.0,...,0.000612,0.283182,0.481425,0.078719,-0.050842,0.995600,0.000799,340394.25,3573621.44,1432.31
1,173.87,66.24,49.25,36056,1,1,1,0,0,5.0,...,0.000751,0.288867,0.386681,0.036121,-0.101415,0.994188,0.001110,340380.87,3573621.84,1433.25
2,159.62,66.21,50.04,34904,1,1,0,0,0,6.0,...,0.000135,0.255970,0.314008,0.011052,-0.155212,0.987819,0.000237,340366.62,3573621.81,1434.04
3,173.88,73.03,49.95,35848,1,1,0,0,0,5.0,...,0.001161,0.207503,0.427400,0.108622,-0.067993,0.991755,0.001825,340380.88,3573628.63,1433.95
4,98.62,65.02,53.71,40408,1,1,0,0,0,8.0,...,0.001029,0.168253,0.433131,0.193586,-0.111395,0.974739,0.001709,340305.62,3573620.62,1437.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543599,113.80,507.12,76.39,36664,1,1,0,0,0,-11.0,...,0.000906,0.264089,0.354787,0.254255,-0.110226,0.960835,0.001462,340320.80,3574062.72,1460.39
543600,112.05,507.28,76.89,34904,1,1,0,0,0,-11.0,...,0.000426,0.292659,0.325105,0.330912,-0.155697,0.930728,0.000689,340319.05,3574062.88,1460.89
543601,106.83,507.32,78.85,34152,1,1,1,0,0,-11.0,...,0.000408,0.310660,0.519154,0.346066,-0.203357,0.915906,0.000492,340313.83,3574062.92,1462.85
543602,107.98,507.22,78.40,32936,1,1,1,0,0,-11.0,...,0.000718,0.301042,0.598865,0.369223,-0.111760,0.922596,0.000797,340314.98,3574062.82,1462.40


In [44]:
# Show ground as surface
ground_delaunay = scipy.spatial.Delaunay(df[['Z','X','Y']])
#ground_surf = p3.plot_trisurf(df.loc[df['Curvature'],'Y'], df.loc[df['ground'],'Z'], df.loc[df['ground'],'X'], ground_delaunay.simplices, color='lightgrey')
#fig.meshes.append(ground_surf)
#ground.visible=False

In [50]:
ground_surf = p3.plot_trisurf(df[['Curvature','Y']], df[['Curvature','Z']], df[['Curvature','X']], ground_delaunay.simplices, color='lightgrey')
fig.meshes.append(ground_surf)
#p3.show()

In [58]:
df['flatness'] = df['Curvature']

In [59]:
fig = p3.figure(width=500)
fig.xlabel='Y'
fig.ylabel='Z'
fig.zlabel='X'
all_points = p3.scatter(df['Y'], df['flatness'], df['X'], color='viridis', size=.5)
p3.squarelim()
p3.show()

In [ ]:
{
            "type": "writers.las",
            "compression": "true",
            "minor_version": "2",
            "dataformat_id": "3",
            "filename":"/home/jose/Documents/LiDAR/TM/GIS_data/mine_locations/mine_loc_0/mine0_color.las"
        }

In [16]:
write = {
    "pipeline": [ TM_las,
        {   "type":"filters.eigenvalues",
            "knn":16},
        {   "type":"filters.normal",
            "knn":16},
                 
        {   "type":"writers.las",
            "filename":py_export+"/outputfile1.las",
            "forward":"Curvature"}
         
    ]}
pipeline = pdal.Pipeline(json.dumps(write))
pipeline.validate()
%time n_points = pipeline.execute()
print('Pipeline selected {} points'.format(n_points))

RuntimeError: writers.las: Error in 'forward' option.  Unknown field for forwarding: 'Curvature'.

In [43]:
write = {
    "pipeline": [
        {   "type":"readers.numpy",
            "filename":"/home/jose/Github/my_projs/abm_detect/Figures/python_export/TM_lidar/numpy_test.npy"},
                 
        {   "type":"writers.las",
            "filename":py_export+"/outputfile1.las"}
         
    ]}
pipeline = pdal.Pipeline(json.dumps(write))
pipeline.validate()
%time n_points = pipeline.execute()
print('Pipeline selected {} points'.format(n_points))


CPU times: user 305 ms, sys: 32.1 ms, total: 338 ms
Wall time: 338 ms
Pipeline selected 543604 points


In [ ]:
[
    "uncolored.las",
    {
      "type":"filters.colorization",
      "dimensions":"Red:1:1.0, Blue, Green::256.0",
      "raster":"aerial.tif"
    },
    "colorized.las"
]


In [40]:
np.save(py_export+'/numpy_test',arr,allow_pickle=False)